<a href="https://colab.research.google.com/github/j54854/myColab/blob/main/pom1_suppl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 生産管理技術1の補助資料

この補助資料では，「生産管理技術1」の第14回講義で扱ったサプライチェーンのブルウィップ効果を発生させる要因の１つを取り上げ，その影響を計算機シミュレーションで確認してみる．

## サプライチェーンのモデル

### チェーンを構成する各段階の在庫管理方式

複数の段階（下の数値例では4段階）で構成されるサプライチェーンを考え，最下流の段階から順に $i=0, 1, 2, ...$ と番号を振る（番号を $0$ から始めるのは，単にPythonを使うのにその方が都合がよいためである）．各段階には在庫があり，それらは注文（情報の流れ）と配送（ものの流れ）で直列に繋がれている．

ある期の注文が上流の段階に届くまでの注文リードタイムを $LTO$ ，ある期に出荷された品物が下流の段階に届くまでの配送リードタイムを $LTT$ とおく．簡単のため，これらのリードタイムは段階によらないものとしよう．このとき，第 $t$ 期末に段階 $i$ から出された注文に対応する品物は，上流の段階 $i+1$ に十分な在庫があれば，第 $t+LTO+LTT+1$ 期首には段階 $i$ に納品されていることになる．

各段階 $i$ は，毎期発注を行う定期発注方式で運用されていると考える．ただし，毎期需要予測を行い，それに基づいて定期発注方式の補充点を更新しているものとしよう．また，需要予測では，需要の平均 $\bar{y}_i$ および分散 $\sigma_i^2$ の予測値をそれぞれ次式で表される指数平滑法で更新しているとする（ただし，$SPA$ および $SPV$ はそれぞれの平滑化係数，$y_{i,t}$ は，段階 $i$ が第 $t$ 期に受注する需要量である） ．

$\bar{y}_i = \bar{y}_i +SPA \times (y_{i,t} -\bar{y}_i)$

$\sigma_i^2 = \sigma_i^2 +SPV \times [(y_{i,t} -\bar{y}_i)^2 -\sigma_i^2]$

これらの予測値を用いると，安全在庫量 $SS_i$ および補充点 $RP_i$ は次式で与えられる（ただし，$SSK$ は安全在庫の係数）．

$SS_i = SSK \times \sigma_i \times \sqrt{LTO+LTT+1}$

$RP_i　= \bar{y}_i \times (LTO+LTT+1) + SS_i $

したがって，各段階では，毎期，この補充点と，発注済み未入荷量を含む有効在庫量の差（補充点-有効在庫量）を発注することになる（ただし，この差が負の値の場合は発注しないものとする）．

### 市場からの需要

続いて，市場（消費者）からの需要の時系列 $y_{0,t}$ をモデル化しよう．これは最下流の段階 $0$ への需要である．不確実性を伴う時系列であれば何でもよいが，ここでは，この需要の時系列は次式で与えられるものとしよう．

$y_{0,t} = BASE + CV \times y_{0,t-1} +\epsilon_t$

ただし，$\epsilon_t$ は，$t$ にかかわらず独立に，平均 $0$，標準偏差 $SD$ の正規分布に従う確率変数である．このとき，定常状態での $y_{0,t}$ の平均および分散は，それぞれ $BASE /(1-CV)$ および $SD^2 /(1-CV^2)$ となる．



## シミュレーションのためのコード

### Marketクラス

このクラスは，上述の市場（消費者）を表現したものである．get_y() メソッドで需要を1期分ずつ生成していくようになっており，それらの値はリスト y に格納されていくため，後で参照することもできる．また，get_ave() と get_var() の2つのメソッドでそれぞれ時系列の平均と分散を確認することができる．

In [2]:
import matplotlib.pyplot as plt
import random

class Market:
    def __init__(self, BASE=100, SD=5, CV=0.5):
        self.BASE = BASE  # ベース値
        self.SD = SD  # 各期の誤差の標準偏差
        self.CV = CV  # 前期との相関を表すパラメータ
        self.y = []  # 需要時系列のリスト（空で初期化）

    def get_ave(self):
        return self.BASE /(1 -self.CV)  # 需要時系列の平均

    def get_var(self):
        return self.SD **2 /(1 -self.CV **2)  # 需要時系列の分散

    def get_y(self, t=None):
        if self.y:
            y_last = self.y[-1]  # 前期の需要量
        else:
            y_last = self.get_ave()  # 前期がなければ平均で代用
        y_next = round(
            self.BASE + self.CV *y_last +random.normalvariate(0, self.SD)
            )
        self.y.append(y_next)  # 需要時系列のリストの末尾に追加
        return self.y[-1]  # リストの末尾の要素を返す

例えば下記のコードで，実際に需要の時系列を生成し，その推移を折れ線グラフで表してみることができる．

In [ ]:
mkt = Market()  # Marketクラスのインスタンスを生成し，mktと名付ける
for i in range(100):  # 100期分の需要を生成
    mkt.get_y()

def my_plot(y, buyer='Market', ub=500):
    plt.figure(num=None, figsize=(12, 8))  # 作図の大きさを指定
    plt.plot(y, marker='o', label=buyer)  # リストyの中身を折れ線グラフでフロットする
    plt.xlabel('Period')
    plt.ylabel('Demand from ' +buyer)
    plt.ylim(0, ub)
    plt.legend()

my_plot(mkt.y)

### AgentクラスとSupplyChainクラス

Agentクラスは，サプライチェーンの各段階を上述の定期発注方式で運用するエージェントを表しており，SupplyChainクラスは，それらを集めたサプライチェーン全体を表現している．

SupplyChainクラスに渡す引数のうち，HZN はシミュレーションを行う期間，LEN はサプライチェーンの段階数，LTO，LTT，SPA，SPV，SSK はそれぞれ上で説明したパラメータである．すべての引数にデフォルト値が指定されているので，明示的に値を指定しなくてもよい．指定しなかった場合は，それらのデフォルト値が使用される．

このSupplyChainクラスのインスタンスを生成した後，その run() メソッドを呼ぶとシミュレーションが実行されるようになっている．





In [4]:
import math

class Agent:
    def __init__(self, chain, i):
        self.chain = chain  # サプライチェーンへのポインタ
        self.stage = i  # 担当ステージの番号
        self.y_ave = self.chain.mkt.get_ave()  # 需要の平均の予測値（真値で初期化）
        self.y_var = self.chain.mkt.get_var()  # 需要の分散の予測値（真値で初期化）
        self.at_hand = self.order_up_to_level() # 手元在庫量(適正値で初期化）
        self.shortage = 0  # 受注済み未出荷量
        self.en_route = 0  # 発注済み未入荷量
        self.order = []  # 発注量のリスト（空で初期化）
        self.flow = []  # 出荷量のリスト（空で初期化）

    def get_parent(self):  # 1つ上流側のステージを返す
        if self.stage >= self.chain.LEN -1:
            return None  # 最上流の場合はNone
        else:
            return self.chain.stages[self.stage +1]

    def get_child(self):  # 1つ下流側のステージを返す
        if self.stage <= 0:
            return None  # 最下流の場合はNone
        else:
            return self.chain.stages[self.stage -1]

    def total_lt(self):  # 総リードタイム
        return self.chain.LTO +self.chain.LTT

    def safty_stock(self):  # 安全在庫量（この段階では浮動小数点で返す）
        lt_and_ct = self.total_lt() +1  # 総リードタイムとサイクルタイムの和
        return math.sqrt(lt_and_ct *self.y_var) *self.chain.SSK

    def order_up_to_level(self):  # 補充点（ここで整数に切り上げる）
        lt_and_ct = self.total_lt() +1  # 総リードタイムとサイクルタイムの和
        return math.ceil(lt_and_ct *self.y_ave +self.safty_stock())

    def receive_flow(self, now):  # 期首に(LTO+LTT+1)期前の発注が補充される
        ordered = now -(self.total_lt() +1)  # 発注した期
        shipped = now -self.chain.LTT -1  # 出荷された期
        if shipped < 0:  # 0期以前の発注は考えない
            return
        if self.get_parent():  # 最上流でなければ，上流からの出荷量が入る
            flow = self.get_parent().flow[shipped]
        else:  # 最上流なら，発注量がそのまま入る
            flow = self.order[ordered]
        self.at_hand += flow  # 手元在庫量を更新
        self.en_route -= flow  # 発注済み未入荷量を更新

    def update_forecast(self, y):
        sq = (y -self.y_ave) **2  # 今期の予測誤差平方
        self.y_ave += (y -self.y_ave) *self.chain.SPA  # 指数平滑法で需要予測平均を更新
        self.y_var += (sq -self.y_var) *self.chain.SPV  # 指数平滑法で需要予測分散を更新

    def send_flow(self, now):
        if self.get_child():  # 最下流でなければ，下流からの発注量に対応する
            ordered =  now -self.chain.LTO  # 発注された期
            if ordered < 0:
                return
            else:
                demand = self.get_child().order[ordered]
        else:  # 最下流なら，市場からの発注量に対応する
            demand = self.chain.mkt.get_y()
        gross_order = demand +self.shortage  # 受注済み未出荷量を加える
        self.flow.append(min(gross_order, self.at_hand))  # 手持ち在庫量が出荷量の上限
        self.at_hand -= self.flow[-1]  # 手持ち在庫量を更新
        self.shortage = gross_order -self.flow[-1]  # 受注済み未出荷量を更新
        self.update_forecast(demand)  # 需要予測を更新
        print('period {}, stage {}: demand = {}'.format(now, self.stage, demand))

    def place_order(self):
        order_quantity = self.order_up_to_level() -(self.at_hand +self.en_route)
        order_quantity = max(0, order_quantity)  # 負の量は発注できないとする
        self.order.append(order_quantity)  # 発注量のリストを更新
        self.en_route += order_quantity  # 発注済み未入荷量の更新

    def operate(self, now):
        self.receive_flow(now)  # 上流ステージからの補充を受ける
        self.send_flow(now)  # 下流ステージからの注文を受けて出荷
        self.place_order()  # 上流ステージへの発注


class SupplyChain:
    def __init__(self, HZN=100, LEN=4, LTO=2, LTT=2, SPA=0.3, SPV=0.3, SSK=1.65):
        self.HZN = HZN  # シミュレーションを行う期間数
        self.LEN = LEN  # サプライチェーンのステージ数
        self.LTO = LTO  # 注文リードタイム
        self.LTT = LTT  # 配送リードタイム
        self.SPA = SPA  # 平均の指数平滑予測の平滑化パラメータ
        self.SPV = SPV  # 分散の指数平滑予測の平滑化パラメータ
        self.SSK = SSK  # 安全在庫量算出のための安全係数
        self.mkt = Market()  # 市場（最終需要）のモデル
        self.stages = []  # 各ステージの運用担当エージェントのリスト
        for i in range(self.LEN):
            self.stages.append(Agent(self, i))

    def run(self):
        for t in range(self.HZN):
            for stage in self.stages:
                stage.operate(t)

## シミュレーションの実行例

それでは，すべてのパラメータの値をデフォルト値のままにして，シミュレーションを実行してみよう．そのためには，下記のコードを実行すればよい．

In [ ]:
chain = SupplyChain()  # パラメータの設定を変えてシミュレーションを実行してみたい場合は，ここで，SupplyChain(LTO=3, LTT=4)などのようにパラメータの値を引数で明示的に与えればよい．
chain.run()

これで100期分の各段階からの発注量（その上流から見ると需要量）がシミュレートできたことになる．ただし，結果が数値として画面に表示されるだけではわかりにくいので，得られた発注量の推移を折れ線グラフで表現してみることにする．


In [ ]:
my_plot(chain.mkt.y, 'Market', 500)

for i in range(4):
    my_plot(chain.stages[i].order, "Stage " +str(i), 500)

上流に行くほど発注量の変動が大きくなっていることが確認できる．これがまさにブルウィップ効果である．

各段階の発注量の分散の大きさも比較しておこう．

In [ ]:
import statistics

print('Variance of Demand from Market = {}'.format(round(statistics.variance(chain.mkt.y))))

for i in range(4):
    print('Variance of Demand from Stage {} = {}'.format(i, round(statistics.variance(chain.stages[i].order))))

## まとめ

サプライチェーンのブルウィップ効果は様々な要因によって複合的に引き起こされる現象である．そして，そうした要因のうちの代表的なものの１つとして，需要予測の短期的な変動がリードタイムに渡って累積され，発注量に反映されてしまうことが挙げられる．

この補助資料では，その要因がブルウィップ効果につながるメカニズムをモデル化し，シミュレーションによって実際にブルウィップ効果が生じることを確認した．

上記のコードを利用すると，パラメータの値を変更したときに，ブルウィップ効果にどのような変化が生じるかを調べてみることもできる．